In [95]:
import string
from langdetect import detect
import lyricsgenius
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
from adjustText import adjust_text
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from scrape_genius import get_lyrics_from_dataframe

In [96]:
df_grammys = pd.read_csv('grammy_df.csv')

In [123]:
get_lyrics_from_dataframe(df_grammys, 'yCZEZPMfq_-XZBsxCS2x1UKScfpa48rnKgTGaHwOsiv6DAzdtFUqCZWdrVQakSb5')

Searching for "Nel Blu Dipinto Di Blu (Volare)"...
Done.
Searching for "Catch A Falling Star"...
Done.
Searching for "Witchcraft (Single)"...
Done.
Searching for "The Chipmunk Song"...
Done.
Searching for "Fever (Single)"...
Done.
Searching for "Mack The Knife"...
Done.
Searching for "The Three Bells"...
Done.
Searching for "Like Young (Single)"...
Done.
Searching for "High Hopes (Single)"...
Done.
Searching for "A Fool Such As I (Single)"...
Done.
Searching for "The Theme From A Summer Place"...
Done.
Searching for "Nice 'N Easy (Single)"...
Done.
Searching for "Mack The Knife (Single)"...
Done.
Searching for "Georgia On My Mind (Single)"...
Done.
Searching for "Are You Lonesome Tonight? (Single)"...
Done.
Searching for "Moon River"...
Done.
Searching for "Up A Lazy River"...
Done.
Searching for "The Second Time Around (Single)"...
Done.
Searching for "Take Five"...
Done.
Searching for "Big Bad John (Single)"...
Done.
Searching for "I Left My Heart In San Francisco"...
Done.
Searching

,title,artist,winner,year,lyrics
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,"Nel blu dipinto di blu (Volare) Lyrics[Testo di ""Nel blu dipinto di blu (Volare)""]\n\n[Strofa 1]\nPenso che un sogno così non ritorni mai più\nMi dipingevo le mani e la faccia di blu\nPoi d'improvviso venivo dal vento rapito\nE incominciavo a volare nel cielo infinito\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\n\n[Strofa 2]\nE volavo, volavo felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano spariva, lontano laggiù\nUna musica dolce suonava soltanto per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu dipinto di blu\nFelice di stare lassù\nYou might also like[Strofa 3]\nMa tutti i sogni nell'alba svaniscon perché\nQuando tramonta la luna li porta con sé\nMa io continuo a sognare negli occhi tuoi belli\nChe sono blu come un cielo trapunto di stelle\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\n\n[Strofa 4]\nE continuo a volare felice\nPiù in alto del sole ed ancora più su\nMentre il mondo pian piano scompare negli occhi tuoi blu\nLa tua voce è una musica dolce che suona per me\n\n[Ritornello]\nVolare, oh oh\nCantare, oh oh oh oh\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù\nNel blu degli occhi tuoi blu\nFelice di stare quaggiù con te1Embed"
1,Catch A Falling Star,Perry Como,0,1958,"Catch a Falling Star Lyrics[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\n\n[Verse 1]\nFor love may come and tap you on the shoulder some starless night\nJust in case you feel you want to hold her\nYou'll have a pocketful of starlight\n(Pocketful of starlight, hm, hm, hm, hm, hm, hm)\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy day\nYou might also like[Hook]\n(Save it for a rainy, save it for a rainy, rainy, rainy, day)\n\n[Bridge]\nFor when your troubles start multiplying and they just might\nIt's easy to forget them without trying\nWith just a pocketful of starlight\n\n[Chorus]\nCatch a falling star and put it in your pocket\nNever let it fade away\nCatch a falling star and put it in your pocket\nSave it for a rainy dayEmbed"
2,Witchcraft (Single),Frank Sinatra,0,1958,"Sweetest Maleficia Lyrics[Gilles de Rais:]\nI conjure you\nBarron, Satan, Beelzebub\nBy the Father, the Son, and the Holy Spirit\nBy the Virgin Mary and all the saints\nTo appear, in person\nSo that you may speak to us\nAnd fulfill our desires\n\nCome at my bidding\nAnd I will grant you\nWhatever you want, however vile\nAnd the containing of my life\n\nHe would rise triumphant\nAll done up\nOn a plume of craven wings\nTrafficking with sycophants\nSharing his cup\nAmidst other graver things\n\nAlchemists and sorcerers stitched his head\nWith the stench of pitch and myrrh\n\nThe devout faded out but the pagan remained\nThe candles burnt low and still nothing came\nBearing golden secrets from a cold malevolent race\nYou might also likeHe would have his demon\nHe would have his vice\nAll save his soul was up for sacrifice\nDespite their raising not a single hair\nEverything stank of witchcraft there\n\nFrom the stained chapel to the statued lawn\nIn Caprineum on the lake\nTo the still lit crypts and the slit of dawn\nSliding down the towers, it all smelt fake\n\nHe needed answers not advice\nIntending to devise\nA lengthy train of torture for the fool\nWho thought a seance would suffice\nOr sighted, furre

In [183]:
merged_df = pd.read_csv('genius_grammy.csv')

In [184]:
merged_df.dropna(inplace=True)

In [185]:
merged_df.title.nunique()

343

In [186]:
merged_df

,title,artist,winner,year,lyrics
0,Nel Blu Dipinto Di Blu (Volare),Domenico Modugno,1,1958,"Nel blu dipinto di blu (Volare) Lyrics[Testo di ""Nel blu dipinto di blu (Volare)""]\n\n[Strofa 1]..."
1,Catch A Falling Star,Perry Como,0,1958,Catch a Falling Star Lyrics[Chorus]\nCatch a falling star and put it in your pocket\nNever let i...
2,Witchcraft (Single),Frank Sinatra,0,1958,"Sweetest Maleficia Lyrics[Gilles de Rais:]\nI conjure you\nBarron, Satan, Beelzebub\nBy the Fath..."
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958,"The Chipmunk song Lyrics[Dave:] Alright you Chipmunks, Ready to sing your song?\n[Alvin:] I'd sa..."
4,Fever (Single),Peggy Lee,0,1958,Jaden Smith Discography Lyrics2012\nFlame (Just Cuz) (SINGLE)\nGive It To Em' (SINGLE)\nThe Cool...
...,...,...,...,...,...
343,Woman,Doja Cat,0,2022,TranslationsTürkçeEspañolPortuguêsItalianoΕλληνικάDeutschFrançaisفارسیPerfect Lyrics[Verse 1]\nI...
344,You And Me On The Rock,Brandi Carlile Featuring Lucius,0,2022,You and Me On The Rock Lyrics[Verse 1]\nThey build wooden houses on frozen ponds\nIn the summert...
345,Good Morning Gorgeous,Mary J. Blige,0,2022,"Good Morning Gorgeous Lyrics[Intro]\nOoh\nOoh, ooh, ohh, ah\nOoh, ooh\n\n[Verse 1]\nIt's so hard..."
346,Easy On Me,Adele,0,2022,TranslationsTürkçeไทย / Phasa ThaiSvenskaEspañolHrvatskiРусскийPortuguês한국어日本語ItalianoMagyarΕλλη...


In [187]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Stem the words (optional)
    porter = nltk.PorterStemmer()
    stemmed_tokens = [porter.stem(token) for token in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)
    return preprocessed_text

merged_df['preprocessed_lyrics'] = merged_df['lyrics'].apply(preprocess_text)


In [188]:
merged_df.drop(['lyrics'], axis=1, inplace=True)

In [189]:
# Identify language of each lyric using langdetect
merged_df['language'] = merged_df['preprocessed_lyrics'].apply(detect)

# Filter out non-English lyrics
df_english = merged_df[merged_df['language'] == 'en']


In [190]:
df_english.dropna(axis=1, inplace=True)

if df_english['preprocessed_lyrics'].dtype != 'object':
    df_english['preprocessed_lyrics'] = df_english['preprocessed_lyrics'].astype(str)

/var/folders/hj/n94x31sj08bdmgwjs6ybxbf40000gn/T/ipykernel_93901/2351279597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english.dropna(axis=1, inplace=True)


In [191]:
df_english

,title,artist,winner,year,preprocessed_lyrics,language
1,Catch A Falling Star,Perry Como,0,1958,catch fall star lyric [ choru ] catch fall star put pocket never let fade away catch fall star p...,en
2,Witchcraft (Single),Frank Sinatra,0,1958,"sweetest maleficia lyric [ gill de rai : ] conjur barron , satan , beelzebub father , son , holi...",en
3,The Chipmunk Song,David Seville And The Chipmunks,0,1958,"chipmunk song lyric [ dave : ] alright chipmunk , readi sing song ? [ alvin : ] 'd say [ theodor...",en
4,Fever (Single),Peggy Lee,0,1958,jaden smith discographi lyrics2012 flame ( cuz ) ( singl ) give em ' ( singl ) coolest ( singl )...,en
5,Mack The Knife,Bobby Darin,1,1959,"mack knife lyricsoh shark , babe , teeth , dear show pearli white jackknif old macheath , babe k...",en
...,...,...,...,...,...,...
343,Woman,Doja Cat,0,2022,translationstürkçeespañolportuguêsitalianoελληνικάdeutschfrançaisفارسیperfect lyric [ vers 1 ] f...,en
344,You And Me On The Rock,Brandi Carlile Featuring Lucius,0,2022,rock lyric [ vers 1 ] build wooden hous frozen pond summertim water 's gone diagon line rolled-o...,en
345,Good Morning Gorgeous,Mary J. Blige,0,2022,"good morn gorgeou lyric [ intro ] ooh ooh , ooh , ohh , ah ooh , ooh [ vers 1 ] 's hard hold ai ...",en
346,Easy On Me,Adele,0,2022,translationstürkçeไทย / phasa thaisvenskaespañolhrvatskiрусскийportuguês한국어日本語italianomagyarελλη...,en


In [192]:
def read_lexicon(filename):
    word_sentiments = {}

    # Parse lexicon file
    with open(filename, 'r') as f:
        for line in f:
            # Ignore blank lines
            if line.strip():
                word, sentiment, value = line.split()
                if value == '1':
                    # Add sentiment to dictionary
                    cur_list = word_sentiments.get(word, [])
                    cur_list.append(sentiment)
                    word_sentiments[word] = cur_list

    return word_sentiments

In [193]:
def get_sentiment_percentages(track_df, sentiment):
    sentiment_pcts = []

    # Parse tracks
    for title in track_df.index.values:
        num_sentiment_words = 0
        all_words = str(track_df.loc[title]['preprocessed_lyrics']).split()

        # Parse words in lyrics
        for word in all_words:
            if sentiment in word_sentiments_dict.get(word, []):
                num_sentiment_words += 1

        sentiment_pcts.append(num_sentiment_words/len(all_words))

    return pd.DataFrame(data=sentiment_pcts, index=track_df.index.values, columns=[sentiment + '_pct'])


In [194]:
#Initialize sentiment dictionary and track dataframe
word_sentiments_dict = read_lexicon('lexicon.txt')


#Generate sentiment percentages based on NRC Lexicon sentiments
sentiments = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']
for sentiment  in sentiments:
    sentiment_df = get_sentiment_percentages(df_english, sentiment)
    df_english = df_english.join(sentiment_df)

In [195]:
# Calculate the weighted average of the sentiment percentages for each row in the DataFrame
weights = [0.1, 0.5, 0.1, 0.5]
sentiments = ['negative_pct', 'sadness_pct', 'disgust_pct', 'fear_pct']
df_english['negative_index'] = (df_english[sentiments] * weights).sum(axis=1)

# Rescale the 'gloom_index' column to a range of 0 to 100
max_gloom_value = df_english['negative_index'].max()
min_gloom_value = df_english['negative_index'].min()
df_english['negative_index'] = (df_english['negative_index'] - min_gloom_value) / (max_gloom_value - min_gloom_value) * 100

# Sort the DataFrame by the 'gloom_index' column in descending order
df_english.sort_values(by='negative_index', ascending=False, inplace=True)



In [196]:
df_english

,title,artist,winner,year,preprocessed_lyrics,language,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,negative_index
178,If I Ever Lose My Faith In You (Single),Sting,0,1993,sorrow ( syrup rmx ) lyricsyea uh huh abstract art first thing put shit 2015 record shit nigga '...,en,0.047511,0.067873,0.052036,0.095023,0.049774,0.117647,0.088235,0.076923,0.033937,0.063348,100.000000
311,Bad Guy,Billie Eilish,1,2019,translationshrvatskitürkçesvenskaespañolрусскийromânăportuguêspolskiitalianohebrewdeutschfrançai...,en,0.074074,0.027778,0.074074,0.074074,0.023148,0.087963,0.023148,0.087963,0.013889,0.023148,94.444444
83,Help Me (Single),Joni Mitchell,0,1974,lilith lyric [ vers 1 ] septemb dont rain dont throw back ocean tear im drown hard breath 'm kne...,en,0.018750,0.050000,0.025000,0.081250,0.037500,0.100000,0.050000,0.081250,0.012500,0.043750,91.071429
241,Not Ready To Make Nice,Dixie Chicks,1,2006,"readi make nice lyric [ vers 1 ] forgiv , sound good forget , 'm sure could say time heal everyt...",en,0.054852,0.054852,0.054852,0.063291,0.037975,0.101266,0.042194,0.084388,0.012658,0.042194,86.895720
2,Witchcraft (Single),Frank Sinatra,0,1958,"sweetest maleficia lyric [ gill de rai : ] conjur barron , satan , beelzebub father , son , holi...",en,0.047138,0.050505,0.060606,0.080808,0.037037,0.097643,0.067340,0.063973,0.016835,0.047138,85.695046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,A Taste Of Honey,Herb Alpert And The Tijuana Brass,1,1965,"tast honey lyric [ intro ] tast honey ! tast much sweeter wine [ vers 1 ] dream first kiss , fee...",en,0.000000,0.030612,0.000000,0.000000,0.020408,0.000000,0.112245,0.000000,0.020408,0.000000,0.000000
81,I Honestly Love You,Olivia Newton-John,1,1974,honestli love lyric [ vers 1 ] mayb hang around littl know got somewher els go got someth tell n...,en,0.000000,0.025210,0.000000,0.000000,0.100840,0.000000,0.100840,0.000000,0.016807,0.000000,0.000000
98,Love Theme From A Star Is Born (Evergreen),Barbra Streisand,0,1977,evergreen ( love theme ’ star born ’ ) lyricslov soft easi chair love fresh morn air one love sh...,en,0.000000,0.103896,0.000000,0.000000,0.181818,0.000000,0.220779,0.000000,0.012987,0.090909,0.000000
50,By The Time I Get To Phoenix,Glen Campbell,0,1967,time get phoenix lyric [ vers 1 ] time get phoenix 'll risin' 'll find note left hangin' door 'l...,en,0.000000,0.078652,0.000000,0.000000,0.011236,0.000000,0.033708,0.000000,0.011236,0.000000,0.000000


In [197]:
df_english.drop_duplicates(['title','artist', 'year'], inplace=True)

In [198]:
df_english.sort_values(['negative_index'], ascending=False).head(20)

,title,artist,winner,year,preprocessed_lyrics,language,anger_pct,anticipation_pct,disgust_pct,fear_pct,joy_pct,negative_pct,positive_pct,sadness_pct,surprise_pct,trust_pct,negative_index
178,If I Ever Lose My Faith In You (Single),Sting,0,1993,sorrow ( syrup rmx ) lyricsyea uh huh abstract art first thing put shit 2015 record shit nigga '...,en,0.047511,0.067873,0.052036,0.095023,0.049774,0.117647,0.088235,0.076923,0.033937,0.063348,100.000000
311,Bad Guy,Billie Eilish,1,2019,translationshrvatskitürkçesvenskaespañolрусскийromânăportuguêspolskiitalianohebrewdeutschfrançai...,en,0.074074,0.027778,0.074074,0.074074,0.023148,0.087963,0.023148,0.087963,0.013889,0.023148,94.444444
83,Help Me (Single),Joni Mitchell,0,1974,lilith lyric [ vers 1 ] septemb dont rain dont throw back ocean tear im drown hard breath 'm kne...,en,0.018750,0.050000,0.025000,0.081250,0.037500,0.100000,0.050000,0.081250,0.012500,0.043750,91.071429
241,Not Ready To Make Nice,Dixie Chicks,1,2006,"readi make nice lyric [ vers 1 ] forgiv , sound good forget , 'm sure could say time heal everyt...",en,0.054852,0.054852,0.054852,0.063291,0.037975,0.101266,0.042194,0.084388,0.012658,0.042194,86.895720
2,Witchcraft (Single),Frank Sinatra,0,1958,"sweetest maleficia lyric [ gill de rai : ] conjur barron , satan , beelzebub father , son , holi...",en,0.047138,0.050505,0.060606,0.080808,0.037037,0.097643,0.067340,0.063973,0.016835,0.047138,85.695046
134,Girls Just Want To Have Fun (Single),Cyndi Lauper,0,1984,"hey ( girl wan na fun ) lyricshey , hey 's matter ? girl wan na fun come hey , hey 's matter ? g...",en,0.003289,0.098684,0.016447,0.072368,0.092105,0.092105,0.105263,0.072368,0.006579,0.016447,80.845865
32,People (Single),Barbra Streisand,0,1964,"peopl peopl [ singl version ] lyric [ choru : dave gahan & martin l. gore ] peopl peopl , get al...",en,0.071642,0.000000,0.062687,0.065672,0.000000,0.071642,0.000000,0.065672,0.005970,0.000000,76.844350
275,Stronger (What Doesn't Kill You),Kelly Clarkson,0,2012,translationsελληνικάstrong ( ’ kill ) lyric [ vers 1 ] know bed feel warmer sleepin ' alon ( sle...,en,0.003205,0.009615,0.000000,0.060897,0.006410,0.064103,0.006410,0.083333,0.009615,0.003205,76.282051
252,Paper Planes,M.I.A,0,2008,"paper plane lyric [ vers 1 ] fli like paper , get high like plane catch border got visa name com...",en,0.076667,0.063333,0.020000,0.060000,0.043333,0.093333,0.056667,0.060000,0.063333,0.050000,69.295238
268,The Cave,Mumford & Sons,0,2011,"cave lyric [ vers 1 ] 's empti valley heart sun , rise slowli walk away fear fault 've left behi...",en,0.033708,0.050562,0.005618,0.050562,0.050562,0.084270,0.084270,0.073034,0.022472,0.078652,68.764045


In [ ]:
df_english.to_csv('english_df.csv', index=False)